# Como realizar el analisis de datos en Jupyter

[PyCon Ar en PyVideo.org](https://pyvideo.org/events/pycon-ar-2011.html)

In [1]:
import pathlib

In [2]:
carpeta = pathlib.Path('../datos')

In [8]:
archivos = [ elemento for elemento in carpeta.glob('*/videos/*.json') ]

In [9]:
len(archivos)

237

In [10]:
import pandas

In [17]:
series = [ pandas.read_json(archivo, typ='series') for archivo in archivos ]

In [21]:
df = pandas.concat(series, axis=1).T

In [24]:
df.speakers.explode()

0            Alan Runyan
1         Alejandro Cura
2         Alejandro Cura
3      Alejandro Pereira
3        Silvestre Huens
             ...        
233        Carol Willing
234          Matias Lang
234     Leonardo Lazzaro
235      Javier Mansilla
236       Claudio Freire
Name: speakers, Length: 270, dtype: object

In [25]:
import bqplot.pyplot as plt
pandas.set_option('plotting.backend', 'bqplot.pyplot')

In [35]:
df.speakers.apply(len).sort_values().plot()
plt.show()

In [36]:
import bqplot

In [37]:
x = df.speakers.explode().value_counts()
x = x[x.between(2,10)]

plt.figure()
plt.scales(scales={'x':bqplot.OrdinalScale(),'y':plt.LinearScale()})
bar = plt.bar(x.index,x.values,orientation='horizontal')
plt.show()

In [40]:
df.recorded = pandas.to_datetime( df.recorded  )
recorded = df.recorded.dt.day.value_counts()
plt.figure()
plt.scatter('index','recorded',data=recorded.reset_index())
plt.xlabel('Anio')
plt.ylabel('Cantidad de charlas')
plt.show()

In [50]:
titulos = ( df.title
    .str.lower()
    .str.replace('\W',' ',regex=True)
    .str.replace('\s+',' ',regex=True)
    .str.translate(str.maketrans('áéíóúñ','aeioun'))
)
titulos

0                                                  plone
1                                   todo sobre deferreds
2                                dbus con el pie derecho
3                   plone 4 tiempo de coccion 30 minutos
4                          robotica con python y arduino
                             ...                        
232    reverse engineering 170 mil lineas de document...
233                     keynote sharing by carol willing
234    sobreviviendo al sindrome del segundo sistema ...
235             testing en proyectos de machine learning
236                 todo lo que no queres saber de async
Name: title, Length: 237, dtype: object

In [ ]:
a = (titulos.str.split(' ',expand=True)
        .stack()
#         .reset_index(drop=True)
        .value_counts()
        .to_frame(name='cuentas')
        .reset_index()
        .rename(columns={'index':'palabra'})
)
